In [13]:
import os
import pandas as pd
import glob

In [18]:
# create path to json files
path_to_json = "Task\Retweet Network\Event"
json_pattern = os.path.join(path_to_json, "*.json")
file_list = glob.glob(json_pattern)
# read json files into dataframes
dfs = []
for file in file_list:
    data = pd.read_json(file, orient="index")
    # store dataframes in a list
    dfs.append(data)
# concatenate all dataframes
df = pd.concat(dfs, ignore_index=True)